In [2]:
import numpy as np
import os
import h5py

import mspt.image_processing as img
import mspt.particle_detection as detect
os.environ['NUMEXPR_MAX_THREADS'] = '32'

# Process movie for display

## Select movie to process

In [ ]:
#file = os.path.normpath(r'C:\Users\admin\Desktop\MSPT test\0001_sample.mp') # set manually
file = img.fileDialog(os.getcwd()) # or via dialog
assert os.path.isfile(file), 'File "{}" does not exist'.format(file)

## Parameter settings

In [ ]:
# General parameters
frame_range = [0,10000] # Restrict analysis to certain frames, e.g. [0, 2000], only applied if all_frames = False
navg = 1 # Frame averaging before background removal

# Background removal
mode = 'continuous_median' # Choose background removal strategy
window_length = 1001 # Set median window length

# Choose between CPU and GPU processing. Set one of the two options True and the other one False.
parallel = True # Use multiple CPU cores to perform background substraction
GPU = False # Use GPU to perform background substraction, fastest option (requires CUDA and pytorch)

## Background removal

In [ ]:
# Apply continuous median background removal
frames, file = img.mp_reader(batch_mode=True,
                             file_to_load=file,
                             frame_range=frame_range,
                             mode=mode,
                             navg=navg,
                             window_length=window_length,
                             parallel=parallel, 
                             GPU=GPU)

# Alternatively, load processed movie for display

## Select processed movie to load

In [9]:
#file = os.path.normpath(r'C:\Users\admin\Desktop\MSPT test\001_sample_median1001.h5') # set manually
file = img.fileDialog(os.getcwd()) # or via dialog
assert os.path.isfile(file), 'File "{}" does not exist'.format(file)

# Load processed movies from HDF5 file
with h5py.File(file, 'r') as h5_file:
    frames = np.asarray(h5_file['frames']).copy()
    window_length = np.asarray(h5_file['window_length']).copy()
    frame_range = np.asarray(h5_file['frame_range']).copy()
    
print('Loaded processed movie {}'.format(file))

Loaded processed movie C:/Users/admin/Desktop/jup diff test/SLB_tetraStrep_0-625nM_50ul_aldolase_200_nM_50ul_R2/001_sample_median1001/001_sample_median1001.h5


# Movie viewer

## Basic viewer with frame slider

In [10]:
# Use this viewer to slide through frames
contrast_range = [-0.005, 0.005] # For display
img.frame_slider(frames, vmin=contrast_range[0], vmax=contrast_range[1])

interactive(children=(IntSlider(value=0, description='frame', max=74999), Output()), _dom_classes=('widget-int…

## Viewer with frame slider and detection threshold selection

In [11]:
# Use this viewer to select a suitable particle detection threshold T1
contrast_range = [-0.005, 0.005] # For display
detect.frame_slider_view_cands_simple_filter(frames, vmin=contrast_range[0], vmax=contrast_range[1])

interactive(children=(IntSlider(value=0, description='frame', max=74999), SelectionSlider(description='T1', op…